# Libraries

In [1]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import scipy
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns
from collections import Counter
import random
from scipy import sparse
from scipy.stats import pearsonr
import re

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import operator

import nltk
from nltk.corpus import stopwords
import torch
from torchtext import data
from torchtext import datasets

In [3]:
! pip install transformers -q
from transformers import pipeline

     |████████████████████████████████| 5.5 MB 4.4 MB/s 
     |████████████████████████████████| 163 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 39.9 MB/s 


In [4]:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

# Recommendations from Keywords

## With Emotions

In [5]:
def cosine_similarity_of(text1, text2):
        
        # taking words
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        # creating dictionary from each word and count
        vector1 = Counter(first)
        vector2 = Counter(second)

        # changing vectors to sets to find comman words with intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            # getting amount of each common word for vectors and multiply them also add them
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        # getting summation of word counts for every vector
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        # calculating magnitude of vector with squared sums
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
            return 0.0
        else:
            return float(dot_product) / magnitude

In [6]:
# Για αυτό το racommend δεν χρησιμοποιείται, αλλά έχει αντικατασταθεί από την get_rating_weight_with_threshold
def rating_weight(rating,Q):
    '''calculating rating weight'''
    w = (2*Q/5)*rating - Q
    return w

In [7]:
def calculate_final_score(cos_sim, w):
# ανάθεση βάρους στο cosine similarity
        effect = (cos_sim / 100) * w
        return cos_sim + effect

In [8]:
def get_rating_weight_with_threshold(rating, count, threshold, Q):
            # rating effect 
            w = (2*Q/5)*rating - Q
            # multiplier effect
            M = math.exp((-threshold*0.68)/count)

            return w * M

In [9]:
#αποθηκεύεται για να χρησιμοποιηθεί για το user interface
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/hotels_for_keywords_recommendation.csv') 

In [10]:
df

,UserId,Name,Review,Rating,Emotion,sentiment,Review_Clean,Reviewer_Count,Restaurant_Count,word_without_stop,positive
0,1,amalia,"amazing hotel, great location and helpful staf...",5,admiration,2,amazing hotel great location and helpful staff...,21,1378,amazing hotel great location helpful staff fac...,True
1,462,herodion,stayed here for 4 nights. arrived late past mi...,5,approval,2,stayed here for 4 nights arrived late past mid...,21,2000,stayed 4 nights arrived late past midnight giv...,True
2,451,herodion,this place is pretty much perfect for those in...,5,admiration,2,this place is pretty much perfect for those in...,21,2000,place pretty much perfect interested acropolis...,True
3,452,herodion,the hotel itself is average. the rooms are sma...,3,admiration,1,the hotel itself is average the rooms are smal...,21,2000,hotel average rooms small halls felt odd said ...,True
4,453,herodion,the lovely and classic hotel herodion pleased ...,5,admiration,2,the lovely and classic hotel herodion pleased ...,21,2000,lovely classic hotel herodion pleased every wa...,True
...,...,...,...,...,...,...,...,...,...,...,...
23766,1672,royal,my husband and i visited the royal olympic hot...,4,approval,2,my husband and i visited the royal olympic hot...,2,2000,husband visited royal olympic hotel end august...,True
23767,1671,royal,hotel is only 10 mins walking time to the hist...,3,admiration,1,hotel is only 10 mins walking time to the hist...,2,2000,hotel 10 mins walking time historical sites ro...,True
23768,1670,royal,make no mistake about it this hotel is a great...,3,admiration,1,make no mistake about it this hotel is a great...,2,2000,make mistake hotel great base see top attracti...,True
23769,1669,royal,i stayed at royal olympic from sunday and chec...,5,approval,2,i stayed at royal olympic from sunday and chec...,2,2000,stayed royal olympic sunday checked wednesday ...,True


In [11]:
def get_recommendations_with_emotion(df,keywords):

        score_dict = {}

        emotion_from_keywords = emotion(keywords)[0]['label']
        print('Emotion is ' + emotion_from_keywords)

        for index, row in df.iterrows():
            cs = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['Rating']
            rating_count = row['Restaurant_Count']
            emotion_from_comment = row['Emotion']


            if row['positive']==True:
                pos_rat = (7+rating)/2
            else:
                pos_rat = rating/2
                
            threshold=110
            # I choose Q as 10 to not to change weight too much
            rating_contribution = get_rating_weight_with_threshold(pos_rat,rating_count,threshold,10)

            score_from_similarity = calculate_final_score(cs, rating_contribution)

            # Αν το συναίσθημα από τα keywords ισοδυναμεί με το συναίσθημα του σχολίου τότε το score αυξάνεται κατά 10%
            if(emotion_from_comment == emotion_from_keywords):
              last_score = score_from_similarity + 0.2*score_from_similarity
            else:
              last_score = score_from_similarity

            score_dict[index] = last_score
            
        #sort τα σχόλια
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        #δημιουργία νέου dataframe για την αποθήκευση των recommendations
        result = pd.DataFrame(columns=('Name', 'score'))

        # τα reviews με τα υψηλότερα cosign similarity
        for i in sorted_scores:
            result = result.append({'Name': df.iloc[i[0]]['Name'],
                                        'score': i[1]}, ignore_index=True)
            
            counter += 1

            if counter>20:
                break

        result = result.drop_duplicates(subset=['Name'], keep='first')
        result = result.reset_index(drop=True)   
        return result

In [13]:
keywords = "happy employers"
recommendations = get_recommendations_with_emotion(df,keywords)
recommendations[:15]

Emotion is joy


,Name,score
0,polis grand,0.296097
1,electra_metropolis,0.291742
2,titania,0.265427
3,melia,0.255728
4,central,0.247261
5,hera,0.243809
6,herodion,0.241953
7,new,0.231023
8,plaka_hotel,0.196461
9,marble,0.192902
